# Setup

In [1]:
!pip install -q pandas numpy scipy

In [2]:
from scipy.stats import norm
import numpy as np
import pandas as pd
from typing import TypeAlias, List, Tuple
import urllib.request

# BO loop

In [3]:
def expected_improvement(mean, std, best):
  '''
  EI implemented as available in this link
  https://ekamperi.github.io/machine%20learning/2021/06/11/acquisition-functions.html#expected-improvement-ei
  '''
  z = (mean - best) / std
  return (mean - best)*norm.cdf(z) + std*norm.pdf(z)

def upper_confidence_bound(mean, std, best):
  '''
  best not used for UCB. Just keeping it here to maintain the same signature for all acquisition functions
  '''
  _alpha = 1.0
  return mean + _alpha*std


In [4]:
class DummyModel:
  '''
  Dummy wrapper for illustrate model training and inference
  '''
  def __init__(self) -> None:
    pass

  def predict(self,
              x: List[str]
              ) -> List[Tuple[float, float]]:
    return [np.random.rand(2) for _ in enumerate(x)]

  def train(self,
            x: List[str],
            y: List[int]
            ) -> object:
    return DummyModel()


def create_dummy_positives():
  '''
  Creating a dummy pool of data with positive examples
  '''
  vocabulary = ['A','R','N','D','C','Q','E','G','H','I', 'L','K','M','F','P','S','T','W','Y','V']
  sequences = [''.join(np.random.choice(vocabulary, size=50)) for _ in range(100)]
  classes = np.random.choice([1], size=100)
  return pd.DataFrame({'sequence': sequences, 'class': classes})


def create_dummy_pool():
  '''
  Creating a dummy pool of data
  '''
  vocabulary = ['A','R','N','D','C','Q','E','G','H','I', 'L','K','M','F','P','S','T','W','Y','V']
  sequences = [''.join(np.random.choice(vocabulary, size=50)) for _ in range(100)]
  # classes = np.random.choice([0, 1], size=100) # No clases for unlabeled data
  return pd.DataFrame(
          {
          'sequence': sequences,
          # 'class': classes
          }
      )

urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/hemo-positive.npz",
    "positive.npz",
)
urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/ml/data/hemo-negative.npz",
    "negative.npz",
)

with np.load("positive.npz") as r:
    pos_data = r[list(r.keys())[0]]
with np.load("negative.npz") as r:
    neg_data = r[list(r.keys())[0]]

def build_fakes(n, data, max_length=200):
    result = []
    for _ in range(n):
        # sample this many subsequences
        avg_data_lengths = np.mean(np.count_nonzero(data, axis=1))
        k = np.clip(np.random.poisson(avg_data_lengths), 1, len(data) - 1)
        from scipy.stats import norm
        k = np.clip(np.random.poisson(1), 0, len(data) - 2) + 2
        idx = np.random.choice(range(len(data)), replace=False, size=k)
        seq = []
        lengths = []
        # cut-up k into one new sequence
        for i in range(k):
            if np.argmin(data[idx[i]]) > 1:
                l = np.ceil(2 * np.random.randint(1, np.argmin(data[idx[i]])) / k).astype(int)
                lengths.append(l)
                j = np.random.randint(0, np.argmin(data[idx[i]]) - lengths[i])
            else:
                lengths.append(1)
                j = 0
            s = data[idx[i]][j:j+lengths[i]]
            seq.append(s)
        seq.append([0] * (len(data[0]) - sum(lengths)))
        # check to make sure seq length >= 1 and is valid
        sample_seq = np.concatenate(seq)
        if np.argmin(np.concatenate(seq)) >= 1:
            result.append(sample_seq)
    return np.array(result)

### Hypothesis:

If the reliable negatives are well selected. The model should perform better --> Proved in the PU paper

### Hypothesis 2:
Can BO be used to robustly select good reliable negatives?

In [5]:
model = DummyModel()
pool_u = create_dummy_pool()
pool_p = create_dummy_positives()

aqfxn = expected_improvement
best = 0
best_acc = 0
n_exps = 5


for i in range(1, n_exps+1):
  N = i*len(pool_u)//n_exps
  pool_u['yhat'] = model.predict(pool_u['sequence'])
  pool_u['score'] = [aqfxn(*y, best) for y in pool_u['yhat']]
  pool_u = pool_u.sort_values(by='score', ascending=False)

  best = pool_u.iloc[0]['score']

  new_dataset = pd.concat([pool_u[:N], pool_p[:N]])
  new_dataset = new_dataset.sample(frac=1).reset_index(drop=True)

  model = model.train(new_dataset['sequence'], new_dataset['class'])


model #Ideally having a good model at this point
